In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from model_xregopose import xR_egoposemodel
from tqdm import tqdm

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=16,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = xR_egoposemodel(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		img = test_batch['image']
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {'image':img}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['pred_pose']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
			# break
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp703'+'/*'))
path_

['/workspace/2d_to_3d/apps/exp703/0.pth',
 '/workspace/2d_to_3d/apps/exp703/1.pth',
 '/workspace/2d_to_3d/apps/exp703/2.pth',
 '/workspace/2d_to_3d/apps/exp703/3.pth',
 '/workspace/2d_to_3d/apps/exp703/4.pth',
 '/workspace/2d_to_3d/apps/exp703/5.pth',
 '/workspace/2d_to_3d/apps/exp703/6.pth',
 '/workspace/2d_to_3d/apps/exp703/7.pth',
 '/workspace/2d_to_3d/apps/exp703/8.pth',
 '/workspace/2d_to_3d/apps/exp703/9.pth',
 '/workspace/2d_to_3d/apps/exp703/best.pth']

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()
raise

RuntimeError: No active exception to reraise

In [5]:
for i,p in enumerate(path_):
	output_path = f'/dataset/data/xr_egopose_depth_sil_2/baseline703_{i}.json'
	main(model_path=p,output_path=output_path)

05:20:00 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use Interpolat

{'FullBody': {'All': 225.26840209960938, 'UpperStretching': 219.24481201171875, 'Walking': 237.12586975097656, 'Reacting': 227.7544403076172, 'LowerStretching': 242.7022247314453, 'Talking': 182.42916870117188, 'Patting': 267.2939453125, 'Gesticuling': 183.41539001464844, 'Gaming': 225.20668029785156, 'Greeting': 195.8561553955078}, 'UpperBody': {'All': 206.4003448486328, 'UpperStretching': 209.54306030273438, 'Walking': 227.46682739257812, 'Reacting': 195.15643310546875, 'LowerStretching': 189.5369110107422, 'Talking': 202.94647216796875, 'Patting': 195.0943145751953, 'Gesticuling': 185.93777465820312, 'Gaming': 190.57247924804688, 'Greeting': 222.23094177246094}, 'LowerBody': {'All': 244.13644409179688, 'UpperStretching': 228.94654846191406, 'Walking': 246.78492736816406, 'Reacting': 260.3524169921875, 'LowerStretching': 295.8675231933594, 'Talking': 161.911865234375, 'Patting': 339.4935607910156, 'Gesticuling': 180.8930206298828, 'Gaming': 259.8408508300781, 'Greeting': 169.48133850

  7%|▋         | 400/5352 [02:26<28:32,  2.89it/s]

{'FullBody': {'All': 224.69683837890625, 'UpperStretching': 218.35301208496094, 'Walking': 234.25782775878906, 'Reacting': 222.69729614257812, 'LowerStretching': 246.18301391601562, 'Talking': 186.40296936035156, 'Patting': 258.7860107421875, 'Gesticuling': 185.0850830078125, 'Gaming': 228.2462615966797, 'Greeting': 198.01031494140625}, 'UpperBody': {'All': 205.7212371826172, 'UpperStretching': 208.8577880859375, 'Walking': 225.86947631835938, 'Reacting': 196.4066619873047, 'LowerStretching': 189.08578491210938, 'Talking': 201.39874267578125, 'Patting': 194.54202270507812, 'Gesticuling': 182.3189239501953, 'Gaming': 191.3578338623047, 'Greeting': 222.93405151367188}, 'LowerBody': {'All': 243.6724395751953, 'UpperStretching': 227.84820556640625, 'Walking': 242.6461639404297, 'Reacting': 248.9879150390625, 'LowerStretching': 303.2801818847656, 'Talking': 171.4071807861328, 'Patting': 323.0299377441406, 'Gesticuling': 187.85122680664062, 'Gaming': 265.1346130371094, 'Greeting': 173.086547

 11%|█         | 600/5352 [03:35<27:29,  2.88it/s]

...

 93%|█████████▎| 5000/5352 [28:57<02:04,  2.82it/s]

{'FullBody': {'All': 143.1293182373047, 'LowerStretching': 149.3936767578125, 'UpperStretching': 143.8896942138672, 'Gesticuling': 112.95475006103516, 'Walking': 144.52407836914062, 'Gaming': 140.26193237304688, 'Patting': 131.04104614257812, 'Talking': 115.83073425292969, 'Reacting': 140.4859161376953, 'Greeting': 117.13658905029297}, 'UpperBody': {'All': 125.23086547851562, 'LowerStretching': 105.53562927246094, 'UpperStretching': 135.23655700683594, 'Gesticuling': 111.25331115722656, 'Walking': 114.32815551757812, 'Gaming': 118.7651138305664, 'Patting': 120.33170318603516, 'Talking': 106.59028625488281, 'Reacting': 123.09405517578125, 'Greeting': 98.03893280029297}, 'LowerBody': {'All': 161.0277862548828, 'LowerStretching': 193.25173950195312, 'UpperStretching': 152.54281616210938, 'Gesticuling': 114.65618896484375, 'Walking': 174.72000122070312, 'Gaming': 161.7587432861328, 'Patting': 141.75038146972656, 'Talking': 125.07119750976562, 'Reacting': 157.8777618408203, 'Greeting': 136.

 97%|█████████▋| 5200/5352 [30:06<00:53,  2.83it/s]

{'FullBody': {'All': 143.1889190673828, 'LowerStretching': 149.47280883789062, 'UpperStretching': 143.90501403808594, 'Gesticuling': 113.07650756835938, 'Walking': 144.65835571289062, 'Gaming': 140.74229431152344, 'Patting': 131.3862762451172, 'Talking': 115.89862823486328, 'Reacting': 140.26272583007812, 'Greeting': 117.08692932128906}, 'UpperBody': {'All': 125.26911163330078, 'LowerStretching': 105.60948181152344, 'UpperStretching': 135.25367736816406, 'Gesticuling': 111.32677459716797, 'Walking': 114.2937240600586, 'Gaming': 118.90000915527344, 'Patting': 120.3704605102539, 'Talking': 106.65328979492188, 'Reacting': 122.76409912109375, 'Greeting': 97.98788452148438}, 'LowerBody': {'All': 161.1087188720703, 'LowerStretching': 193.33612060546875, 'UpperStretching': 152.5563507080078, 'Gesticuling': 114.82624053955078, 'Walking': 175.02296447753906, 'Gaming': 162.58456420898438, 'Patting': 142.40208435058594, 'Talking': 125.14395141601562, 'Reacting': 157.76136779785156, 'Greeting': 13

100%|██████████| 5352/5352 [30:59<00:00,  2.88it/s]
11:01:37 [INFO] Main: Saving evaluation results...
11:01:37 [INFO] Main: Done.
